# Проект по SQL

## Описание проекта
Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. <br>
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. <br>
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта. 

## Цель проекта
Проанализировать базу данных сервиса чтения книг, чтобы сформулировать ценностное предложение для нового продукта

## Задачи проекта
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Описание данных

### Таблица `books`
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

### Таблица `authors`
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

### Таблица `publishers`
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

### Таблица `ratings`
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

### Таблица `reviews`
Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Выполнение проекта

### Импорт библиотек и установка связи

In [9]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [10]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Установка функции для удобства вывода таблиц

In [12]:
def run_query(query: str) -> pd.DataFrame:
    with engine.connect() as con:
        return pd.read_sql(sql=text(query), con=con)

### 1. Посчитайте, сколько книг вышло после 1 января 2000 года

In [14]:
query_1 = """
    SELECT COUNT(*)
    FROM books
    WHERE publication_date > '2000-01-01';
"""
result_1 = run_query(query_1)
result_1

,count
0,819


#### Вывод по пункту "1. Посчитайте, сколько книг вышло после 1 января 2000 года"
После 1 января 2000 года вышло **819 книг**

### 2. Для каждой книги посчитайте количество обзоров и среднюю оценку

In [17]:
query_2 = """
    SELECT 
        b.book_id,
        b.title,
        COUNT(DISTINCT r.review_id) AS review_count,
        AVG(rt.rating) AS average_rating
    FROM books b
    LEFT JOIN reviews r ON b.book_id = r.book_id
    LEFT JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY b.book_id, b.title;
"""
result_2 = run_query(query_2)
result_2

,book_id,title,review_count,average_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


#### Вывод для пункта "2. Для каждой книги посчитайте количество обзоров и среднюю оценку"
Для каждой книги были посчитайты количество обзоров и средняя оценка, столбцы названы как полагается

### 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [20]:
query_3 = """
    SELECT 
        p.publisher,
        COUNT(*) AS book_count
    FROM books b
    JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY book_count DESC
    LIMIT 1;
"""
result_3 = run_query(query_3)
result_3

,publisher,book_count
0,Penguin Books,42


#### Вывод по пункту "3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры"
Издательство `"Penguin Books"` наибольшее число книг толще 50 страниц – всего данное издательство выпустило **42** такие **книги**. На втором месте находится издательство `"Vintage"`, выпустив **31 книгу**, тройку лидеров завершает издательство `"Grand Central Publishing"` – **25 книг**

### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [23]:
query_4 = """
    WITH books_with_50_ratings AS (
        SELECT 
            b.book_id,
            b.author_id
        FROM books b
        JOIN ratings rt ON b.book_id = rt.book_id
        GROUP BY b.book_id, b.author_id
        HAVING COUNT(rt.rating) >= 50
    )
    SELECT 
        a.author,
        ROUND(AVG(rt.rating), 3) AS average_rating
    FROM books_with_50_ratings bw
    JOIN authors a ON bw.author_id = a.author_id
    JOIN ratings rt ON bw.book_id = rt.book_id
    GROUP BY a.author
    ORDER BY average_rating DESC
    LIMIT 1;
"""
result_4 = run_query(query_4)
result_4


,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287


#### Вывод по пункту "4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками"
Автором с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, является **`J.K. Rowling/Mary GrandPré`**

### 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [26]:
query_5 = """
    WITH user_rating_count AS (
        SELECT 
            username,
            COUNT(*) AS rating_count
        FROM ratings
        GROUP BY username
        HAVING COUNT(*) > 48
    )
    SELECT AVG(review_count) AS avg_reviews_per_user
    FROM (
        SELECT 
            urc.username,
            COUNT(r.review_id) AS review_count
        FROM user_rating_count urc
        LEFT JOIN reviews r ON urc.username = r.username
        GROUP BY urc.username
    ) AS user_reviews;
"""
result_5 = run_query(query_5)
result_5

,avg_reviews_per_user
0,24.0


#### Вывод по пункту "5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок"
Пользователи, которые поставили больше 48 оценок, в среднем совершают **24 обзора**

## Общий вывод
### Вывод по пункту "1. Посчитайте, сколько книг вышло после 1 января 2000 года"
После 1 января 2000 года вышло **819 книг**

### Вывод для пункта "2. Для каждой книги посчитайте количество обзоров и среднюю оценку"
Для каждой книги были посчитайты количество обзоров и средняя оценка, столбцы названы как полагается

### Вывод по пункту "3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры"
Издательство `"Penguin Books"` наибольшее число книг толще 50 страниц – всего данное издательство выпустило **42** такие **книги**. На втором месте находится издательство `"Vintage"`, выпустив **31 книгу**, тройку лидеров завершает издательство `"Grand Central Publishing"` – **25 книг**

### Вывод по пункту "4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками"
Автором с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, является **`J.K. Rowling/Mary GrandPré`**

### Вывод по пункту "5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок"
Пользователи, которые поставили больше 48 оценок, в среднем совершают **24 обзора**

### Общая рекомендация
Можно запустить сервис для книголюбов, предлагающий подборки книг от лучших издательств, таких как **`Penguin Books`** (по возможности ещё **`Vintage`**, **`Grand Central Publishing`**), и авторов с высокими оценками, например, **`J.K. Rowling/Mary GrandPré`** (как топ-1 автор среди книг с 50 и более оценками). Необходимо уделить внимание активным пользователям, предоставляя персонализированные рекомендации и возможности для обмена мнениями, исключая брошюры и фокусируясь на полноценных изданиях, так как после 1 января 2000 года вышло **819 книг**